# 시퀀스-투-시퀀스 (Sequencd-to-Sequence, seq2seq)


시퀀스-투-시퀀스(Sequence-to-Sequence)는 입력된 시퀀스로부터 다른 도메인의 시퀀스를 출력하는 다양한 분야에서 사용되는 모델입니다. 예를 들어 챗봇(Chatbot)과 기계 번역(Machine Translation)이 그러한 대표적인 예인데, 입력 시퀀스와 출력 시퀀스를 각각 질문과 대답으로 구성하면 챗봇으로 만들 수 있고, 입력 시퀀스와 출력 시퀀스를 각각 입력 문장과 번역 문장으로 만들면 번역기로 만들 수 있습니다.

## 1) 시퀀스 - 투 - 시퀀스  (Sequence-to-Sequence)

<img src="https://github.com/KyoHoon998/hello-world/blob/master/images/1.PNG?raw=true" width = "30%">

위의 그림은 seq2seq 모델로 만들어진 번역기가 'I am a student'라는 영어 문장을 입력받아서, 'je suis étudiant'라는 프랑스 문장을 출력하는 모습을 보여줍니다. 그렇다면, seq2seq 모델 내부의 모습을 알아봅시다.

<img src="https://github.com/KyoHoon998/hello-world/blob/master/images/2.PNG?raw=true" width = "60%"> <br>
seq2seq는 크게 두 개로 구성된 아키텍처로 구성되는데, 바로 인코더와 디코더입니다. 인코더는 입력 문장의 모든 단어들을 순차적으로 입력받은 뒤에 마지막에 이 모든 단어 정보들을 압축해서 하나의 벡터로 만드는데, 이를 컨텍스트 벡터(context vector)라고 합니다. 입력 문장의 정보가 하나의 컨텍스트 벡터로 모두 압축되면 인코더는 컨텍스트 벡터를 디코더로 전송합니다. 디코더는 컨텍스트 벡터를 받아서 번역된 단어를 한 개씩 순차적으로 출력합니다.<br>
<img src="https://github.com/KyoHoon998/hello-world/blob/master/images/3.PNG?raw=true" width = "290">

컨텍스트 벡터에 대해서는 뒤에서 다시 언급하겠습니다만, 위의 그림에서는 컨텍스트 벡터를 4의 사이즈로 표현하였습니다. 그러나 실제로 사용되는 seq2seq 모델에서는 보통 수백 이상의 차원을 갖고있습니다. 이제 인코더와 디코더의 내부를 좀 더 확대해보겠습니다.<br>
<img src= "https://github.com/KyoHoon998/hello-world/blob/master/images/4.PNG?raw=true" width = "600">

인코더 아키텍처와 디코더 아키텍처의 내부는 사실 두 개의 LSTM 아키텍처 입니다. 입력 문장을 받는 LSTM 셀을 인코더라고 하고, 출력 문장을 출력하는 LSTM 셀을 디코더라고 합니다. 이번 챕터에서는 인코더의 LSTM 셀을 주황색으로, 디코더의 LSTM 셀을 초록색으로 표현합니다. 우선 인코더를 자세히보면, 입력 문장은 단어 토큰화를 통해서 단어 단위로 쪼개지고 단어 토큰 각각은 LSTM 셀의 각 시점의 입력이 됩니다. 인코더 LSTM 셀은 모든 단어를 입력받은 뒤에 인코더 LSTM 셀의 마지막 시점의 은닉 상태를 디코더 LSTM 셀로 넘겨주는데 이를 컨텍스트 벡터라고 합니다. 컨텍스트 벡터는 디코더 LSTM 셀의 첫번째 은닉 상태로 사용됩니다.

디코더는 기본적으로 RNNLM(RNN Language Model)입니다.
디코더는 초기 입력으로 문장의 시작을 의미하는 심볼 `<sos>` 가 들어갑니다. 디코더는 `<sos>`가 입력되면, 다음에 등장할 확률이 높은 단어를 예측합니다. 첫번째 시점(time step)의 디코더 LSTM 셀은 다음에 등장할 단어로 je를 예측하였습니다. 첫번째 시점의 디코더 LSTM 셀은 예측된 단어 je를 다음 시점의 LSTM 셀의 입력으로 입력합니다. 그리고 두번째 시점의 디코더 LSTM 셀은 입력된 단어 je로부터 다시 다음에 올 단어인 suis를 예측하고, 또 다시 이것을 다음 시점의 LSTM 셀의 입력으로 보냅니다. 디코더는 이런 식으로 기본적으로 다음에 올 단어를 예측하고, 그 예측한 단어를 다음 시점의 LSTM 셀의 입력으로 넣는 행위를 반복합니다. 이 행위는 문장의 끝을 의미하는 심볼인 `<eos>`가 다음 단어로 예측될 때까지 반복됩니다. 지금 설명하는 것은 테스트 과정 동안의 이야기입니다. 

seq2seq는 훈련 과정과 테스트 과정(또는 실제 번역기를 사람이 쓸 때)의 작동 방식이 조금 다릅니다. 훈련 과정에서는 디코더에게 인코더가 보낸 컨텍스트 벡터와 실제 정답인 상황인 `<sos>` je suis étudiant를 입력 받았을 때, je suis étudiant `<eos>`가 나와야 된다고 정답을 알려주면서 훈련합니다. 이에 대해서는 뒤에 교사 강요(teacher forcing)를 설명하면서 다시 언급하겠습니다. 반면 테스트 과정에서는 앞서 설명한 과정과 같이 디코더는 오직 컨텍스트 벡터와 <go>만을 입력으로 받은 후에 다음에 올 단어를 예측하고, 그 단어를 다음 시점의 LSTM 셀의 입력으로 넣는 행위를 반복합니다. 즉, 앞서 설명한 과정과 위의 그림은 테스트 과정에 해당됩니다. 이번에는 입, 출력에 쓰이는 단어 토큰들이 있는 부분을 좀 더 확대해보겠습니다. <br>
<img src  = "https://github.com/KyoHoon998/hello-world/blob/master/images/5.PNG?raw=true" width = "700">

기계는 텍스트보다 숫자를 잘 처리합니다. 그리고 자연어 처리에서 텍스트를 벡터로 바꾸는 방법으로 워드 임베딩이 사용됍니다. 즉, seq2seq에서 사용되는 모든 단어들은 워드 임베딩을 통해 임베딩 벡터로서 표현된 임베딩 벡터입니다. 위 그림은 모든 단어에 대해서 임베딩 과정을 거치게 하는 단계인 임베딩 층(embedding layer)의 모습을 보여줍니다. <br>
<img src = "https://github.com/KyoHoon998/hello-world/blob/master/images/6.PNG?raw=true" width = "700">

예를 들어 I, am, a, student라는 단어들에 대한 임베딩 벡터는 위와 같은 모습을 가집니다. 여기서는 그림으로 표현하고자 사이즈를 4로 하였지만, 보통 실제 임베딩 벡터는 수백 개의 차원을 가질 수 있습니다. 이제 LSTM 셀에 대해서 확대해보겠습니다.

하나의 LSTM 셀은 각각의 시점(time step)마다 두 개의 입력을 받습니다. <br>
<img src = "https://github.com/KyoHoon998/hello-world/blob/master/images/7.PNG?raw=true" width= "700"><br>
현재 시점(time step)을 t라고 할 때, LSTM 셀은 t-1에서의 은닉 상태와 t에서의 입력 벡터를 입력으로 받고, t에서의 은닉 상태를 만듭니다. 이때 t에서의 은닉 상태는 바로 위에 또 다른 은닉층이나 출력층이 존재할 경우에는 위의 층으로 보내거나, 필요없으면 값을 무시할 수 있습니다. 그리고 LSTM 셀은 다음 시점에 해당하는 t+1의 LSTM 셀의 입력으로 현재 t에서의 은닉 상태를 입력으로 보냅니다.

이런 구조에서 현재 시점 t에서의 은닉 상태는 과거 시점의 동일한 LSTM 셀에서의 모든 은닉 상태의 값들의 영향을 누적해서 받아온 값이라고 할 수 있습니다. 그렇기 때문에 앞서 우리가 언급했던 컨텍스트 벡터는 사실 인코더에서의 마지막 LSTM 셀의 은닉 상태값을 말하는 것이며, 이는 입력 문장의 모든 단어 토큰들의 정보를 요약해서 담고있다고 할 수 있습니다.

디코더는 인코더의 마지막 LSTM 셀의 은닉 상태인 컨텍스트 벡터를 첫번째 은닉 상태의 값으로 사용합니다. 디코더의 첫번째 LSTM 셀은 이 첫번째 은닉 상태의 값과, 현재 t에서의 입력값인`<sos>`로부터, 다음에 등장할 단어를 예측합니다. 그리고 이 예측된 단어는 다음 시점인 t+1 LSTM에서의 입력값이 되고, 이 t+1에서의 LSTM 또한 이 입력값과 t에서의 은닉 상태로부터 t+1에서의 출력 벡터. 즉, 또 다시 다음에 등장할 단어를 예측하게 될 것입니다. 이제 디코더가 다음에 등장할 단어를 예측하는 부분을 확대해보도록 하겠습니다. <br>
<img src = "https://github.com/KyoHoon998/hello-world/blob/master/images/8.PNG?raw=true" width = "400"><br>
출력 단어로 나올 수 있는 단어들은 다양한 단어들이 있습니다. seq2seq 모델은 선택될 수 있는 모든 단어들로부터 하나의 단어를 골라서 예측해야 합니다. 이를 예측하기 위해서 쓸 수 있는 함수로는 뭐가 있을까요? 바로 소프트맥스 함수입니다. 디코더에서 각 시점(time step)의 LSTM 셀에서 출력 벡터가 나오면, 해당 벡터는 소프트맥스 함수를 통해 출력 시퀀스의 각 단어별 확률값을 반환하고, 디코더는 출력 단어를 결정합니다.


